# Complete SDR Email System with Reply Handling

This notebook combines:
1. **Cold email SDR functionality** - Generating & sending cold emails
2. **Reply handling** - Responding to prospect replies automatically

## Architecture

**OUTBOUND (Cold Email):**
```
Sales Manager → 3 Sales Agents → Pick Best → Email Manager → Send
```

**INBOUND (Reply Handling):**
```
Prospect Replies → Webhook → Intent Analyzer → SDR Response Agent → Send Reply
```

## Prerequisites

Create a `.env` file with:
```
SENDGRID_API_KEY=SG.your_api_key_here
OPENAI_API_KEY=sk-your_openai_key_here
```

## ⚠️ Important Note

The OpenAI Agents SDK has issues running in Jupyter notebooks due to async context conflicts. 
This notebook provides wrapper functions to handle this. If you encounter async errors, 
run the code as a Python script instead.

---
## Section 1: Setup & SSL Fix

**⚠️ RUN THIS CELL FIRST** - Fixes SSL certificate issues on Windows

In [ ]:
# =============================================================
# SSL FIX - MUST BE AT THE VERY TOP BEFORE OTHER IMPORTS
# =============================================================
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
os.environ['REQUESTS_CA_BUNDLE'] = certifi.where()

print("✓ SSL certificates configured")
print(f"  Using: {certifi.where()}")

---
## Section 2: Imports

In [ ]:
import os
import json
import hashlib
import asyncio
import re
from datetime import datetime
from typing import Dict, List, Optional
from dataclasses import dataclass, asdict

from dotenv import load_dotenv
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content, ReplyTo
from agents import Agent, Runner, trace, function_tool

# Load environment variables
load_dotenv(override=True)

print("✓ All imports successful!")

---
## Section 3: Configuration

**⚠️ Update these values with your own email addresses!**

In [ ]:
CONFIG = {
    # Your verified SendGrid sender email
    "verified_sender": "ajitpcapde@gmail.com",  # <-- CHANGE THIS
    
    # Recipient email for testing
    "test_recipient": "ajittgosavii@gmail.com",  # <-- CHANGE THIS
    
    # Company information
    "company_name": "ComplAI",
    "company_description": "a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI",
    
    # SDR name for email signatures
    "sdr_name": "Alex",
    
    # Max conversation turns
    "max_conversation_turns": 10,
}

print("=" * 50)
print("Configuration")
print("=" * 50)
print(f"Sender: {CONFIG['verified_sender']}")
print(f"Recipient: {CONFIG['test_recipient']}")
print(f"Company: {CONFIG['company_name']}")
print(f"SDR Name: {CONFIG['sdr_name']}")
print("=" * 50)

---
## Section 4: Test SendGrid Connection

In [ ]:
def send_test_email():
    """Send a test email to verify SendGrid is working."""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    mail = Mail(
        Email(CONFIG["verified_sender"]),
        To(CONFIG["test_recipient"]),
        "Test Email from SDR System",
        Content("text/plain", "SendGrid is working!")
    ).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(f"📧 Status: {response.status_code} (202 = success)")
    return response.status_code

# Uncomment to test:
# send_test_email()

---
## Section 5: Sales Agents (3 Different Styles)

In [ ]:
# Professional agent - serious, formal tone
sales_agent_professional = Agent(
    name="Professional Sales Agent",
    instructions=f"""You are a sales agent working for {CONFIG['company_name']}, 
which provides {CONFIG['company_description']}. 
You write professional, serious cold emails.""",
    model="gpt-4o-mini"
)

# Engaging agent - witty, humorous tone
sales_agent_engaging = Agent(
    name="Engaging Sales Agent",
    instructions=f"""You are a humorous, engaging sales agent working for {CONFIG['company_name']}, 
which provides {CONFIG['company_description']}. 
You write witty, engaging cold emails that are likely to get a response.""",
    model="gpt-4o-mini"
)

# Concise agent - short, to-the-point
sales_agent_concise = Agent(
    name="Concise Sales Agent",
    instructions=f"""You are a busy sales agent working for {CONFIG['company_name']}, 
which provides {CONFIG['company_description']}. 
You write concise, to-the-point cold emails.""",
    model="gpt-4o-mini"
)

print("✓ Sales Agents created")

---
## Section 6: Email Sending Tools

In [ ]:
@function_tool
def send_email(body: str) -> Dict[str, str]:
    """Send a plain text email to sales prospects."""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    mail = Mail(
        Email(CONFIG["verified_sender"], CONFIG["sdr_name"]),
        To(CONFIG["test_recipient"]),
        "Sales Email from ComplAI",
        Content("text/plain", body)
    ).get()
    mail["reply_to"] = {"email": CONFIG["verified_sender"], "name": CONFIG["sdr_name"]}
    sg.client.mail.send.post(request_body=mail)
    print(f"📧 Email sent to {CONFIG['test_recipient']}")
    return {"status": "success"}

@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send an HTML email with subject and body."""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    mail = Mail(
        Email(CONFIG["verified_sender"], CONFIG["sdr_name"]),
        To(CONFIG["test_recipient"]),
        subject,
        Content("text/html", html_body)
    ).get()
    mail["reply_to"] = {"email": CONFIG["verified_sender"], "name": CONFIG["sdr_name"]}
    sg.client.mail.send.post(request_body=mail)
    print(f"📧 HTML email sent to {CONFIG['test_recipient']}")
    return {"status": "success"}

print("✓ Email tools created")

---
## Section 7: Convert Agents to Tools

In [ ]:
tool_professional = sales_agent_professional.as_tool(
    tool_name="professional_agent",
    tool_description="Write a professional cold sales email"
)

tool_engaging = sales_agent_engaging.as_tool(
    tool_name="engaging_agent",
    tool_description="Write an engaging cold sales email"
)

tool_concise = sales_agent_concise.as_tool(
    tool_name="concise_agent",
    tool_description="Write a concise cold sales email"
)

print("✓ Agents converted to tools")

---
## Section 8: Email Formatting Agents

In [ ]:
subject_writer = Agent(
    name="Email Subject Writer",
    instructions="Write compelling, short subject lines for cold sales emails.",
    model="gpt-4o-mini"
)
subject_tool = subject_writer.as_tool(
    tool_name="subject_writer",
    tool_description="Write a subject line"
)

html_converter = Agent(
    name="HTML Converter",
    instructions="Convert text emails to clean, professional HTML.",
    model="gpt-4o-mini"
)
html_tool = html_converter.as_tool(
    tool_name="html_converter",
    tool_description="Convert text to HTML"
)

print("✓ Formatting agents created")

---
## Section 9: Email Manager Agent (Handoff Target)

In [ ]:
emailer_agent = Agent(
    name="Email Manager",
    instructions="""Format and send emails:
1. Use subject_writer to create a subject
2. Use html_converter to format as HTML
3. Use send_html_email to send""",
    tools=[subject_tool, html_tool, send_html_email],
    model="gpt-4o-mini",
    handoff_description="Format and send an email"
)

print("✓ Email Manager created")

---
## Section 10: Sales Manager Agent (Main Orchestrator)

In [ ]:
sales_manager = Agent(
    name="Sales Manager",
    instructions=f"""You are a Sales Manager at {CONFIG['company_name']}.

Steps:
1. Use all 3 agent tools to generate different email drafts
2. Pick the single best email
3. Hand off to Email Manager to format and send

Rules:
- Use the agent tools (don't write emails yourself)
- Hand off exactly ONE email
- Sign from {CONFIG['sdr_name']}""",
    tools=[tool_professional, tool_engaging, tool_concise],
    handoffs=[emailer_agent],
    model="gpt-4o-mini"
)

print("✓ Sales Manager created")

---
## Section 11: Run Cold Email Demo

**Check trace at:** https://platform.openai.com/traces

In [ ]:
def run_cold_email_demo():
    """Run the cold email demo using asyncio.run() to avoid Jupyter async issues."""
    
    async def _run():
        message = f"Send a cold email to 'Dear CEO' from {CONFIG['sdr_name']}"
        print("🚀 Starting Cold Email Demo...")
        print(f"Message: {message}")
        print("\nTrace: https://platform.openai.com/traces")
        print("=" * 50)
        
        with trace("Cold Email Demo"):
            result = await Runner.run(sales_manager, message)
        
        print("\n" + "=" * 50)
        print("✅ Done! Check your email.")
        return result
    
    return asyncio.run(_run())

# Run the demo
result = run_cold_email_demo()

---
---
# PART 2: REPLY HANDLING SYSTEM
---

---
## Section 12: Conversation Storage

In [ ]:
@dataclass
class EmailMessage:
    message_id: str
    from_email: str
    to_email: str
    subject: str
    body: str
    timestamp: str
    direction: str  # "inbound" or "outbound"

@dataclass
class Conversation:
    conversation_id: str
    prospect_email: str
    prospect_name: Optional[str]
    subject: str
    messages: List[Dict]
    status: str  # "active", "meeting_scheduled", "not_interested"
    created_at: str
    updated_at: str

# In-memory storage
conversations: Dict[str, Conversation] = {}

def get_conversation_id(email: str, subject: str) -> str:
    clean_subject = subject.lower()
    for prefix in ['re:', 'fwd:', 'fw:']:
        clean_subject = clean_subject.replace(prefix, '').strip()
    return hashlib.md5(f"{email.lower()}:{clean_subject}".encode()).hexdigest()[:12]

def get_or_create_conversation(email: str, subject: str, name: str = None) -> Conversation:
    conv_id = get_conversation_id(email, subject)
    if conv_id not in conversations:
        conversations[conv_id] = Conversation(
            conversation_id=conv_id,
            prospect_email=email,
            prospect_name=name,
            subject=subject,
            messages=[],
            status="active",
            created_at=datetime.now().isoformat(),
            updated_at=datetime.now().isoformat()
        )
    return conversations[conv_id]

def add_message(conv_id: str, message: EmailMessage):
    if conv_id in conversations:
        conversations[conv_id].messages.append(asdict(message))
        conversations[conv_id].updated_at = datetime.now().isoformat()

print("✓ Conversation storage initialized")

---
## Section 13: Reply Handling Agents

In [ ]:
# Intent Analyzer
intent_analyzer = Agent(
    name="Intent Analyzer",
    instructions="""Analyze the email and respond with ONLY one word:
- INTERESTED
- NOT_INTERESTED  
- MEETING_READY
- NEUTRAL""",
    model="gpt-4o-mini"
)

# SDR Response Agent
sdr_response_agent = Agent(
    name="SDR Response Agent",
    instructions=f"""You are {CONFIG['sdr_name']}, an SDR at {CONFIG['company_name']}.
{CONFIG['company_name']} provides {CONFIG['company_description']}.

Guidelines:
1. Be conversational, not robotic
2. Address their specific points
3. Work toward scheduling a meeting
4. Keep responses to 3-5 sentences
5. Sign off as {CONFIG['sdr_name']}""",
    model="gpt-4o-mini"
)

print("✓ Reply agents created")

---
## Section 14: Send Reply Function

In [ ]:
def send_reply_email(to_email: str, subject: str, body: str) -> Dict[str, str]:
    """Send a reply email."""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    reply_subject = subject if subject.lower().startswith('re:') else f"Re: {subject}"
    
    mail = Mail(
        Email(CONFIG["verified_sender"], CONFIG["sdr_name"]),
        To(to_email),
        reply_subject,
        Content("text/plain", body)
    )
    mail.reply_to = ReplyTo(CONFIG["verified_sender"], CONFIG["sdr_name"])
    
    try:
        response = sg.client.mail.send.post(request_body=mail.get())
        return {"status": "success", "code": str(response.status_code)}
    except Exception as e:
        return {"status": "error", "error": str(e)}

print("✓ Reply function created")

---
## Section 15: Process Incoming Email

In [ ]:
async def process_incoming_email_async(
    from_email: str,
    subject: str,
    body: str,
    sender_name: str = None,
    send_response: bool = False
) -> str:
    """Process an incoming email and generate AI response."""
    
    print("\n" + "=" * 50)
    print(f"📧 FROM: {sender_name or 'Unknown'} <{from_email}>")
    print(f"📋 SUBJECT: {subject}")
    print(f"📝 BODY:\n{body}")
    print("=" * 50)
    
    # Get/create conversation
    conv = get_or_create_conversation(from_email, subject, sender_name)
    
    # Store incoming message
    add_message(conv.conversation_id, EmailMessage(
        message_id=f"in_{datetime.now().timestamp()}",
        from_email=from_email,
        to_email=CONFIG["verified_sender"],
        subject=subject,
        body=body,
        timestamp=datetime.now().isoformat(),
        direction="inbound"
    ))
    
    # Build history
    history = "\n".join([f"[{m['direction'].upper()}] {m['body']}" for m in conv.messages[-6:]])
    
    # Analyze intent
    print("\n🔍 Analyzing intent...")
    with trace("Analyze Intent"):
        intent_result = await Runner.run(intent_analyzer, f"Analyze:\n{body}")
        intent = intent_result.final_output.strip().upper()
    print(f"🎯 Intent: {intent}")
    
    # Update status
    if "NOT_INTERESTED" in intent:
        conv.status = "not_interested"
    elif "MEETING" in intent:
        conv.status = "meeting_scheduled"
    
    # Generate response
    print("\n✍️ Generating response...")
    with trace("Generate Response"):
        response = await Runner.run(
            sdr_response_agent,
            f"History:\n{history}\n\nLatest email:\n{body}\n\nIntent: {intent}\n\nWrite response:"
        )
        reply = response.final_output
    
    print("\n" + "-" * 40)
    print("📤 RESPONSE:")
    print("-" * 40)
    print(reply)
    print("-" * 40)
    
    # Send if enabled
    if send_response:
        result = send_reply_email(from_email, subject, reply)
        if result["status"] == "success":
            add_message(conv.conversation_id, EmailMessage(
                message_id=f"out_{datetime.now().timestamp()}",
                from_email=CONFIG["verified_sender"],
                to_email=from_email,
                subject=f"Re: {subject}",
                body=reply,
                timestamp=datetime.now().isoformat(),
                direction="outbound"
            ))
            print("\n✅ Response sent!")
        else:
            print(f"\n❌ Failed: {result.get('error')}")
    else:
        print("\n⏸️ Response NOT sent (send_response=False)")
    
    return reply

# Wrapper for Jupyter
def process_incoming_email(from_email, subject, body, sender_name=None, send_response=False):
    """Synchronous wrapper for process_incoming_email_async."""
    return asyncio.run(process_incoming_email_async(
        from_email, subject, body, sender_name, send_response
    ))

print("✓ Process email function created")

---
## Section 16: Test Reply Handling

Set `send_response=True` to actually send replies.

In [ ]:
# Test 1: Interested Prospect
print("\n🧪 TEST 1: Interested Prospect")
response1 = process_incoming_email(
    from_email="sarah@techstartup.com",
    subject="Re: ComplAI - SOC2 Solution",
    body="""Hi,

Thanks for reaching out! We're preparing for SOC2 and struggling with documentation.

How does ComplAI handle policy generation?

Best,
Sarah""",
    sender_name="Sarah Chen",
    send_response=False
)

In [ ]:
# Test 2: Not Interested
print("\n🧪 TEST 2: Not Interested")
response2 = process_incoming_email(
    from_email="mike@company.com",
    subject="Re: ComplAI Demo",
    body="""Hi,

Thanks but we're not looking at new tools right now.

Mike""",
    sender_name="Mike",
    send_response=False
)

In [ ]:
# Test 3: Ready for Meeting
print("\n🧪 TEST 3: Ready for Meeting")
response3 = process_incoming_email(
    from_email="jennifer@corp.com",
    subject="Re: SOC2 Compliance",
    body="""This sounds great!

I'd love a demo. Free Thursday or Friday.

Thanks!
Jennifer""",
    sender_name="Jennifer",
    send_response=False
)

---
## Section 17: View Conversations

In [ ]:
print("\n" + "=" * 50)
print("📁 ALL CONVERSATIONS")
print("=" * 50)

for conv_id, conv in conversations.items():
    print(f"\n{conv_id}:")
    print(f"  Prospect: {conv.prospect_name or 'Unknown'} <{conv.prospect_email}>")
    print(f"  Subject: {conv.subject}")
    print(f"  Status: {conv.status}")
    print(f"  Messages: {len(conv.messages)}")

---
## Section 18: Webhook Server (Production)

To receive real emails:
1. Save this as a Python file
2. Run: `python webhook_server.py`
3. Use ngrok: `ngrok http 8000`
4. Configure SendGrid Inbound Parse with ngrok URL

In [ ]:
def create_webhook_server():
    """Create FastAPI webhook server."""
    from fastapi import FastAPI, Form, BackgroundTasks
    from fastapi.responses import JSONResponse
    
    app = FastAPI(title="SDR Webhook")
    
    @app.get("/")
    async def root():
        return {"status": "running", "webhook": "/webhook/inbound"}
    
    @app.post("/webhook/inbound")
    async def handle_inbound(
        background_tasks: BackgroundTasks,
        from_: str = Form(None, alias="from"),
        subject: str = Form(None),
        text: str = Form(None),
        html: str = Form(None),
    ):
        # Parse sender
        sender_email = from_ or ""
        sender_name = None
        if from_ and '<' in from_:
            parts = from_.split('<')
            sender_name = parts[0].strip().strip('"')
            sender_email = parts[1].strip('>')
        
        body = text or (re.sub('<[^<]+?>', '', html) if html else "")
        
        background_tasks.add_task(
            process_incoming_email_async,
            sender_email, subject or "(No Subject)", body, sender_name, True
        )
        
        return JSONResponse(status_code=200, content={"status": "accepted"})
    
    @app.get("/conversations")
    async def list_convs():
        return {cid: {"email": c.prospect_email, "status": c.status, "msgs": len(c.messages)}
                for cid, c in conversations.items()}
    
    return app

print("✓ Webhook server function created")
print("")
print("To run: uvicorn notebook:create_webhook_server --factory --port 8000")

---
## Summary

### What We Built

**Cold Email System:**
- 3 Sales Agents (professional, engaging, concise)
- Sales Manager (coordinator)
- Email Manager (formatter & sender)

**Reply Handling System:**
- Conversation tracking
- Intent Analyzer
- SDR Response Agent
- Webhook server

### Agentic Design Patterns

1. **Tool Use**: `@function_tool` decorator
2. **Agent-as-Tool**: `.as_tool()` method
3. **Handoffs**: Passing control between agents
4. **Orchestration**: Sales Manager coordinating agents